### 使用说明
1. 输入所需信息：
    * working file的文件名(带后缀)，如```PwQ_Pre-Allocation W34_Planners_WorkingFile - Copy.xlsm```，以及sheet_name如```2920```
    * excel属于你工作内容的起始行，如PIC为Yanqing的工作内容从excel中的第```8```行开始
    * 用于计算温饱线的excel文件名称(带后缀)，如```AGL needs W34.xlsx```，以及sheet_name如```Stock needed```
2. 运行本文件，在 [Cell] 中点击 Run All，会自动生成两个文件：
    * ```_pre_allocation_res.xlsx``` 储存计算结果
    * ```_pre_allocation_log.log```储存计算日志以信息查询与比对


In [21]:
# 注：使用前请先确保以下信息填写正确：
working_file_name = 'copy PwQ_Pre-Allocation W40_Planners_WorkingFile - YQ.xlsm'
working_file_sheet_name = '2920'
start_row = 8
agl_needs_file_name = 'AGL needs W40.xlsx'
agl_needs_file_sheet_name = 'Stock needed'
pic = 'Yanqing'

In [22]:
import pandas as pd
import numpy as np
from pre_allocation import Solution
import logging
import xlsxwriter
from datetime import date
from openpyxl import load_workbook
from openpyxl.formula.translate import Translator

In [23]:
Log_Format = "%(levelname)s %(asctime)s - %(message)s"
today = date.today()
today = today.strftime("%Y-%m-%d")
file_name = today + '_pre_allocation'
logging.basicConfig(filename = str(today) + "_pre_allocation_log.log",
                    encoding = "utf-8",
                    filemode = "w",
                    format = Log_Format, 
                    level = logging.INFO)
logger = logging.getLogger()

In [24]:
working_file = pd.read_excel(working_file_name, sheet_name=working_file_sheet_name).iloc[5:, 1:]
working_file.columns = working_file.iloc[0]
working_file = working_file[working_file['PIC'] == pic]
working_file.index = np.arange(start_row, len(working_file) + start_row)
columns = ['Material', 'PIC', 'Prepack', 'SOH', 'Avg Ind.Req (M)','KR(1200) -', 'Direct KR', 'HK(1220) -', 'Direct HK','AU(1180) -', 'Direct AU', 'TW(1280) -', 'Direct TW', 'CN(1190) -',
       'Direct CN', 'JP(1230) -', 'Direct JP', 'TH(1340) -', 'Direct TH'] 
in_transit_columns = working_file.columns[[15,16,17,18]].tolist() # late, Wn, Wn+1, Wn+2 in transit value
columns.extend(in_transit_columns)
working_file = working_file[columns].replace(np.nan, 0)
working_file = working_file.astype({'Avg Ind.Req (M)':int})
for week in in_transit_columns:
    working_file = working_file.astype({week: int})

In [25]:
working_file.head(20)

5,Material,PIC,Prepack,SOH,Avg Ind.Req (M),KR(1200) -,Direct KR,HK(1220) -,Direct HK,AU(1180) -,...,CN(1190) -,Direct CN,JP(1230) -,Direct JP,TH(1340) -,Direct TH,Late,W40,W41,W42
8,C099600006,Yanqing,30,169,67,0,0,0,0,0,...,0,0,120,0,0,0,0,0,0,0
9,C099600229,Yanqing,12,131,36,24,0,36,0,0,...,0,0,0,0,0,0,0,0,0,0
10,C099600345,Yanqing,42,1121,76,126,0,0,0,0,...,336,0,588,0,0,0,0,0,0,0
11,C099600416,Yanqing,64,1868,610,384,0,0,768,0,...,0,0,640,0,0,0,0,0,0,0
12,C099600470,Yanqing,30,372,328,0,0,358,0,0,...,0,0,0,0,660,0,770,0,0,0
13,C099600561,Yanqing,35,1152,152,210,0,105,0,0,...,12075,0,0,0,140,0,0,0,0,0
14,C099600750,Yanqing,12,886,142,0,0,48,0,0,...,432,0,1584,0,0,0,0,0,0,0
15,C099600861,Yanqing,20,4899,3582,640,0,640,0,0,...,57200,0,1400,0,0,0,0,0,2000,17400
16,C299600034,Yanqing,25,30,14,0,0,0,0,0,...,0,0,0,0,0,0,150,0,0,25
17,C299600229,Yanqing,24,8,6,0,0,24,0,0,...,0,0,24,0,0,0,0,0,0,0


In [26]:
newdf = working_file
newdf['KR(1200) -'] = np.where(newdf['KR(1200) -'] - newdf['Direct KR'] <= 0, 0, newdf['KR(1200) -'] - newdf['Direct KR'])
newdf['HK(1220) -'] = np.where(newdf['HK(1220) -'] - newdf['Direct HK'] <= 0, 0, newdf['HK(1220) -'] - newdf['Direct HK'])
newdf['AU(1180) -'] = np.where(newdf['AU(1180) -'] - newdf['Direct AU'] <= 0, 0, newdf['AU(1180) -'] - newdf['Direct AU'])
newdf['TW(1280) -'] = np.where(newdf['TW(1280) -'] - newdf['Direct TW'] <= 0, 0, newdf['TW(1280) -'] - newdf['Direct TW'])
newdf['CN(1190) -'] = np.where(newdf['CN(1190) -'] - newdf['Direct CN'] <= 0, 0, newdf['CN(1190) -'] - newdf['Direct CN'])
newdf['JP(1230) -'] = np.where(newdf['JP(1230) -'] - newdf['Direct JP'] <= 0, 0, newdf['JP(1230) -'] - newdf['Direct JP'])
newdf['TH(1340) -'] = np.where(newdf['TH(1340) -'] - newdf['Direct TH'] <= 0, 0, newdf['TH(1340) -'] - newdf['Direct TH'])
newdf['after DD'] = np.where(newdf['SOH'] - newdf['Avg Ind.Req (M)'] <= 0, 0, newdf['SOH'] - newdf['Avg Ind.Req (M)'])
late, wn, wn1, wn2 = in_transit_columns
till_wn1 = late + '/' + wn + '/' + wn1
newdf[till_wn1] = newdf[late] + newdf[wn] + newdf[wn1]
newdf.drop(['Direct KR','Direct HK','Direct AU','Direct TW','Direct CN','Direct JP','Direct TH',late,wn,wn1], axis=1,inplace=True)

In [27]:
newdf.head(20)

5,Material,PIC,Prepack,SOH,Avg Ind.Req (M),KR(1200) -,HK(1220) -,AU(1180) -,TW(1280) -,CN(1190) -,JP(1230) -,TH(1340) -,W42,after DD,Late/W40/W41
8,C099600006,Yanqing,30,169,67,0,0,0,30,0,120,0,0,102,0
9,C099600229,Yanqing,12,131,36,24,36,0,36,0,0,0,0,95,0
10,C099600345,Yanqing,42,1121,76,126,0,0,84,336,588,0,0,1045,0
11,C099600416,Yanqing,64,1868,610,384,0,0,768,0,640,0,0,1258,0
12,C099600470,Yanqing,30,372,328,0,358,0,0,0,0,660,0,44,770
13,C099600561,Yanqing,35,1152,152,210,105,0,70,12075,0,140,0,1000,0
14,C099600750,Yanqing,12,886,142,0,48,0,0,432,1584,0,0,744,0
15,C099600861,Yanqing,20,4899,3582,640,640,0,520,57200,1400,0,17400,1317,2000
16,C299600034,Yanqing,25,30,14,0,0,0,25,0,0,0,25,16,150
17,C299600229,Yanqing,24,8,6,0,24,0,24,0,24,0,0,2,0


In [28]:
skuList = list(working_file['Material'])
workbook = load_workbook(filename=agl_needs_file_name)
worksheet = workbook[agl_needs_file_sheet_name]
formula_row = 3
for sku in skuList:
    for col in ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
        cell = col + str(formula_row)
        if col == 'A':
            worksheet[cell] = sku
        else:
            formula = worksheet[col+'3'].value
            worksheet[cell] = Translator(formula, origin=col+'3').translate_formula(cell)
    formula_row += 1 
workbook.save("agl_needs_updated.xlsx")

In [29]:
# 神奇bug，openpyxl保存的excel并未真正保存 -> https://codeantenna.com/a/iRcZiE5TVp
# 需要用win32com模拟打开并关闭excel来完成一次储存操作
# https://bbs.huaweicloud.com/blogs/363106 -> win32com安装，仅限windows
# pip install pypiwin32 或 pip install pywin32
# !!!以下命令仅用于windows系统，mac只能手动打开excel保存后关闭，然后跳过该代码继续执行

# 如需运行以下代码，取消注释即可

from win32com.client import Dispatch
def open_save(filename):
    xlApp = Dispatch('Excel.Application')
    xlApp.Visible = False
    xlBook = xlApp.Workbooks.Open(filename)
    xlBook.Save()
    xlBook.Close()
open_save(r"C:\Users\yawang\Desktop\supply_planning_trivial_matters\pre_allocation\agl_needs_updated.xlsx")

In [30]:
agl_needs = pd.read_excel("agl_needs_updated.xlsx", sheet_name=agl_needs_file_sheet_name).iloc[1:len(skuList)+2].fillna(0).rename({'SKU':'Material'}, axis=1)
agl_needs.drop(['Designation'], axis=1, inplace=True)
agl_needs = agl_needs.astype({'KR':int, 'HK': int, 'AU': int, 'TW':int, 'CN': int, 'JP': int, 'TH': int})
print(len(agl_needs))
agl_needs.head(20)

35


,Material,KR,HK,AU,TW,CN,JP,TH
1,C099600006,0,0,0,0,0,0,0
2,C099600229,0,0,0,0,0,0,0
3,C099600345,0,0,0,0,-895,0,0
4,C099600416,0,0,0,0,0,0,0
5,C099600470,0,0,0,0,0,0,0
6,C099600561,0,0,0,0,-9723,0,0
7,C099600750,0,0,0,0,-4108,0,0
8,C099600861,0,0,0,0,-78307,0,0
9,C299600034,0,0,0,0,0,0,0
10,C299600229,0,0,0,0,0,0,0


In [31]:
df = pd.merge(newdf, agl_needs, on='Material', how='outer')
df.drop(['SOH','Avg Ind.Req (M)'], axis=1, inplace=True)
df.insert(3, 'after DD', df.pop('after DD'))
df.insert(4, till_wn1, df.pop(till_wn1))
df.insert(5, wn2, df.pop(wn2))
df.index = np.arange(start_row, len(df) + start_row)
df.head(20)

,Material,PIC,Prepack,after DD,Late/W40/W41,W42,KR(1200) -,HK(1220) -,AU(1180) -,TW(1280) -,CN(1190) -,JP(1230) -,TH(1340) -,KR,HK,AU,TW,CN,JP,TH
8,C099600006,Yanqing,30,102,0,0,0,0,0,30,0,120,0,0,0,0,0,0,0,0
9,C099600229,Yanqing,12,95,0,0,24,36,0,36,0,0,0,0,0,0,0,0,0,0
10,C099600345,Yanqing,42,1045,0,0,126,0,0,84,336,588,0,0,0,0,0,-895,0,0
11,C099600416,Yanqing,64,1258,0,0,384,0,0,768,0,640,0,0,0,0,0,0,0,0
12,C099600470,Yanqing,30,44,770,0,0,358,0,0,0,0,660,0,0,0,0,0,0,0
13,C099600561,Yanqing,35,1000,0,0,210,105,0,70,12075,0,140,0,0,0,0,-9723,0,0
14,C099600750,Yanqing,12,744,0,0,0,48,0,0,432,1584,0,0,0,0,0,-4108,0,0
15,C099600861,Yanqing,20,1317,2000,17400,640,640,0,520,57200,1400,0,0,0,0,0,-78307,0,0
16,C299600034,Yanqing,25,16,150,25,0,0,0,25,0,0,0,0,0,0,0,0,0,0
17,C299600229,Yanqing,24,2,0,0,0,24,0,24,0,24,0,0,0,0,0,0,0,0


In [32]:
def getFinalResult(row):
    prepack = row['Prepack']
    own = row['after DD']
    needs = [row['KR'], row['HK'], row['AU'], row['TW'], row['CN'], row['JP'], row['TH']]
    wants = [row['KR(1200) -'], row['HK(1220) -'], row['AU(1180) -'], row['TW(1280) -'], row['CN(1190) -'], row['JP(1230) -'], row['TH(1340) -']]
    in_transit = [[wn1, row[till_wn1]], [wn2, row[wn2]]]
    s = Solution(needs, wants, prepack, own, in_transit)
    res = s.getDistribution()
    logger.info(f"{row['Material']} ==> \n{s.explanation}\n")
    newCols = ['KR(1200) +', 'HK(1220) +', 'AU(1180) +', 'TW(1280) +', 'CN(1190) +', 'JP(1230) +', 'TH(1340) +', 'comments']
    for i in range(len(newCols)):
        if i == len(newCols) - 1:
            row[newCols[i]] = s.note
        else:
            row[newCols[i]] = int(res[i])

    return row

In [33]:
res = df.apply(getFinalResult, axis=1)

In [34]:
res.head(20)

,Material,PIC,Prepack,after DD,Late/W40/W41,W42,KR(1200) -,HK(1220) -,AU(1180) -,TW(1280) -,...,JP,TH,KR(1200) +,HK(1220) +,AU(1180) +,TW(1280) +,CN(1190) +,JP(1230) +,TH(1340) +,comments
8,C099600006,Yanqing,30,102,0,0,0,0,0,30,...,0,0,0,0,0,30,0,60,0,
9,C099600229,Yanqing,12,95,0,0,24,36,0,36,...,0,0,24,36,0,24,0,0,0,
10,C099600345,Yanqing,42,1045,0,0,126,0,0,84,...,0,0,84,0,0,84,336,504,0,
11,C099600416,Yanqing,64,1258,0,0,384,0,0,768,...,0,0,256,0,0,512,0,448,0,
12,C099600470,Yanqing,30,44,770,0,0,358,0,0,...,0,0,0,30,0,0,0,0,0,
13,C099600561,Yanqing,35,1000,0,0,210,105,0,70,...,0,0,0,0,0,0,980,0,0,
14,C099600750,Yanqing,12,744,0,0,0,48,0,0,...,0,0,0,0,0,0,432,300,0,
15,C099600861,Yanqing,20,1317,2000,17400,640,640,0,520,...,0,0,0,0,0,0,1300,0,0,
16,C299600034,Yanqing,25,16,150,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,av W41
17,C299600229,Yanqing,24,2,0,0,0,24,0,24,...,0,0,0,0,0,0,0,0,0,


In [35]:
writer = pd.ExcelWriter('pre_allocation_res.xlsx', engine='xlsxwriter')
res.to_excel(writer, sheet_name='Sheet1')
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

bottom_row = str(len(res) + 1)
fotmat_basic_info = ['#FFFACD', 'D2:G' + bottom_row]
format_wants = ['#FFC7CE', 'H2:N' + bottom_row]
format_needs = ['#CCFFFF', 'O2:U' + bottom_row]
format_res = ['#E5FFCC', 'V2:AB' + bottom_row]
format_list = [fotmat_basic_info, format_wants, format_needs, format_res] # 涂色，每个format信息为[color, left_top:right_bottom]
for each_format in format_list:
    fm = workbook.add_format({'bg_color': each_format[0]})
    area = each_format[1]
    worksheet.conditional_format(area, {'type': 'no_blanks','format': fm})
worksheet.freeze_panes(1,5)
writer.save()

In [36]:
# 似乎是windows特有的问题，如果不shutdown，无法重写或删除log日志
logging.shutdown()

#### 以下代码仅用于单测

In [37]:
prepack = 69
own = 47000
needs = [0,0,-13,-37,-765,0,0]
wants = [0,0,16,0,828, 0, 0, 0]
in_transit = [["W41", 100], ["W42", 1000]]
s = Solution(needs, wants, prepack, own, in_transit)
print(s.getDistribution())
print(s.explanation)
print(s.note)

[0, 0, 16, 0, 828, 0, 0, 0]
1. prepack=69, 库存量=47000, 各地区声明需求=[0, 0, 16, 0, 828, 0, 0, 0], 满足分配. 剩余库存46156.
 
ok
